In [1]:
import os
import sys
from glob import glob
from tqdm import tqdm
import math
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score

import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader
import albumentations as A
sys.path.insert(0, '../')
from utils import load_json, save_pickle
from model import VanillaEfficientNet, load_model
from dataset import TrainDataset, EvalDataset
from torchvision import transforms
from transform_settings import configure_transform
from albumentations import CenterCrop, Blur, Cutout, Equalize, GaussianBlur, GaussNoise, GlassBlur, GridDistortion, Lambda, MedianBlur, MotionBlur, Normalize, RandomBrightnessContrast, RandomFog, Solarize
from albumentations.pytorch.transforms import ToTensor

# Pairing

In [33]:
def cutmix(batch, alpha):
    data, targets = batch

    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)

    image_h, image_w = data.shape[2:]
    cx = np.random.uniform(0, image_w)
    cy = np.random.uniform(0, image_h)
    w = image_w * np.sqrt(1 - lam)
    h = image_h * np.sqrt(1 - lam)
    x0 = int(np.round(max(cx - w / 2, 0)))
    x1 = int(np.round(min(cx + w / 2, image_w)))
    y0 = int(np.round(max(cy - h / 2, 0)))
    y1 = int(np.round(min(cy + h / 2, image_h)))

    data[:, :, y0:y1, x0:x1] = shuffled_data[:, :, y0:y1, x0:x1]
    targets = (targets, shuffled_targets, lam)

    return data, targets


class CutMixCollator:
    def __init__(self, alpha):
        self.alpha = alpha

    def __call__(self, batch):
        batch = torch.utils.data.dataloader.default_collate(batch)
        batch = cutmix(batch, self.alpha)
        return batch


class CutMixCriterion:
    def __init__(self, reduction):
        self.criterion = nn.CrossEntropyLoss(reduction=reduction)

    def __call__(self, preds, targets):
        targets1, targets2, lam = targets
        return lam * self.criterion(
            preds, targets1) + (1 - lam) * self.criterion(preds, targets2)

In [6]:
transform = transforms.Compose([
    transforms.Resize((512, 384), Image.BILINEAR),
    transforms.CenterCrop((384, 384)),
    # transforms.RandomResizedCrop((224, 224)),
    ])

data_config = {
    'root': '../preprocessed_stratified/train', 
    'transform': transform, 
    'task': 'main',
    'meta_path': '../preprocessed_stratified/metadata.json'
    }
dataset = TrainDataset(**data_config)
model = VanillaEfficientNet(n_class=18)
loader = DataLoader(dataset, batch_size=4)

Loaded pretrained weights for efficientnet-b3


In [103]:
def get_class_ageg_gender(x):
    name = os.path.basename(x)
    gender = name.split('_')[1]
    try:
        age = int(name.split('_')[-2])
    except:
        age = int(name.split('_')[-3])

    CLASSES = {('young', 'female'): 0, ('young', 'male'): 1, ('middle', 'female'): 2, ('middle', 'male'):3, ('old', 'female'): 4, ('old', 'male'): 5}
    if age < 30:
        age = 'young'
    elif age >= 30 and age < 60:
        age = 'middle'
    else:
        age = 'old'
    
    if gender == 'female':
        gender = 'female'
    else:
        gender = 'male'

    return CLASSES[(age, gender)] 

In [130]:
img_paths = glob('../preprocessed_stratified/train/*')
metadata  = metadata = load_json('../preprocessed/metadata.json')
train_imgs = pd.DataFrame(dict(path=img_paths))
train_imgs['class'] = train_imgs['path'].apply(lambda x: get_class_ageg_gender(x))
train_imgs['class'].value_counts()

2    4298
0    3850
1    2891
3    2156
4     581
5     441
Name: class, dtype: int64

In [131]:
TARGET_NUM = 5000


In [132]:
augmentation_nums = 5000 - train_imgs['class'].value_counts()
augmentation_nums = {i:j for i, j in zip(augmentation_nums.index.tolist(), augmentation_nums.tolist())}

In [133]:
ROOT = '../input/data/train/images'

In [140]:
CLASS = 0
N = augmentation_nums[CLASS]
np.random.seed(42)

for _ in range(N):
    sample = np.random.choice(train_imgs[train_imgs['class'] == CLASS]['path'].tolist(), size=2, replace=True).tolist()
    s1 = np.array(Image.open(sample[0]))
    s2 = np.array(Image.open(sample[1]))

    s1_half = s1[:, :192, :]
    s2_half = s2[:, 192:, :]

    paired = np.hstack([s1_half, s2_half])
    break


# Face Detection

In [145]:
import numpy as np
import cv2
import dlib

ModuleNotFoundError: No module named 'dlib'

In [ ]:
!apt-get install cmake Y

In [ ]:
def dlib_to_cv_bounding_box(box):
    # convert dlib bounding box for OpenCV display
    x = box.left()
    y = box.top()
    w = box.right() - x
    h = box.bottom() - y

    return x, y, w, h

def landmarks_to_numpy(landmarks):
    # initialize the matrix of (x, y)-coordinates with a row for each landmark
    coords = np.zeros((landmarks.num_parts, 2), dtype=int)
    # convert each landmark to (x, y)
    for i in range(0, landmarks.num_parts):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)
    # return the array of (x, y)-coordinates
    return coords

def detect_landmarks(img):
    points = []
    detected_faces = frontal_face_detector(img, 1)
    for face in detected_faces:
        landmarks = landmarks_predictor(img, face)
        points.append(landmarks_to_numpy(landmarks))
 
    return landmarks, points, detected_faces, img